<a href="https://colab.research.google.com/github/Linaqruf/sd-notebook-collection/blob/main/Infinite_Image_Browsing_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Infinite Image Browsing Demo**

Project Page : [Here!](https://github.com/zanllp/sd-webui-infinite-image-browsing)

In [ ]:
#@title ##**Install**
import os
from IPython.display import clear_output
from google.colab import drive

root_dir = "/content"
repo_url = "https://github.com/zanllp/sd-webui-infinite-image-browsing.git"
repo_dir = os.path.join(root_dir, "sd-webui-infinite-image-browsing")

mount_drive = False #@param {type:"boolean"}
# @markdown Specify where your `images_dir` located, it also automatically create new directory if couldn't find it.
images_dir = "/content/images" #@param {type:"string"}

if not images_dir:
    images_dir = os.path.join(root_dir, "images")

def clone(url, dir):
    !git clone {url} {dir}

def install_deps():
    os.chdir(repo_dir)
    !pip install -r requirements.txt
    !pip install -U gallery-dl
    clear_output(wait=True)

def main():
    os.chdir(root_dir)

    if mount_drive:
        if not os.path.exists("/content/drive"):
            drive.mount("/content/drive")

    !apt update -yqq
    !apt install aria2 -yqq
  
    clone(repo_url, repo_dir)
    
    os.chdir(repo_dir)
    install_deps()

    os.makedirs(images_dir, exist_ok=True)

    clear_output(wait=True)
    print("All is done! go to the next step.")

main()


In [ ]:
#@title ## **Image Scraper**
import os
import html
from IPython.utils import capture
%store -r

os.chdir(root_dir)
#@markdown Use `gallery-dl` to scrape images from an imageboard site. Specify the `prompt(s)` by separating them with commas, e.g., `hito_komoru, touhou`.
booru = "Danbooru" #@param ["Danbooru", "Gelbooru", "Safebooru"]
prompt = "hito_komoru" #@param {type: "string"}
#@markdown  You can also specify a `custom_url` instead of using a predefined site.
custom_url = "" #@param {type: "string"}
user_agent = "gdl/1.24.5"
#@markdown You can limit the number of images to download by using the `--range` option followed by the desired range. For example `1-200`.
range = "1-200" #@param {type: "string"}
additional_arguments = "--filename /O --no-part"

tags = prompt.split(',')
tags = '+'.join(tags)

replacement_dict = {" ": "", "(": "%28", ")": "%29", ":": "%3a"}
tags = ''.join(replacement_dict.get(c, c) for c in tags)

if booru == "Danbooru":
    url = "https://danbooru.donmai.us/posts?tags={}".format(tags)
elif booru == "Gelbooru":
    url = "https://gelbooru.com/index.php?page=post&s=list&tags={}".format(tags)
else:
    url = "https://safebooru.org/index.php?page=post&s=list&tags={}".format(tags)

valid_url = custom_url if custom_url else url

def scrape(config):
    args = ""
    for k, v in config.items():
        if k.startswith("_"):
            args += f'"{v}" '
        elif isinstance(v, str):
            args += f'--{k}="{v}" '
        elif isinstance(v, bool) and v:
            args += f"--{k} "
        elif isinstance(v, float) and not isinstance(v, bool):
            args += f"--{k}={v} "
        elif isinstance(v, int) and not isinstance(v, bool):
            args += f"--{k}={v} "

    return args

get_url_config = {
    "_valid_url" : valid_url,
    "get-urls" : True,
    "range" : range if range else None,
    "user-agent" : user_agent
}

get_url_args = scrape(get_url_config)
scraper_text = os.path.join(root_dir, "scrape_this.txt")

with capture.capture_output() as cap:
    !gallery-dl {get_url_args} {additional_arguments}
with open(scraper_text, "w") as f:
    f.write(cap.stdout)

os.chdir(images_dir)
!aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -i {scraper_text}

clear_output(wait=True)
print("All is done! go to the next step.")

In [ ]:
#@title **Launch!**
import json
import portpicker
from threading import Thread
from google.colab.output import serve_kernel_port_as_iframe

config = {
    "outdir_txt2img_samples": images_dir,
}

main_app = os.path.join(repo_dir, "app.py")
config_file = os.path.join(root_dir, "config.json")
port = portpicker.pick_unused_port()
window_height = 550 #@param {type:"slider", min:0, max:1000, step:1}

def write_file(filename, config):
    with open(filename, 'w',) as f:
        json.dump(config, f, indent=4)

def run_app():
    !python {main_app} --port={port} --sd_webui_config={config_file}

def main():
    os.chdir(images_dir)

    write_file(config_file, config)

    thread = Thread(target=run_app)
    thread.start()

    serve_kernel_port_as_iframe(port, width='100%', height=window_height, cache_in_notebook=False)

main()